In [4]:
import pandas as pd

import sagemaker
from sagemaker import get_execution_role
import json
import boto3

[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Set Up

In [3]:
sess = sagemaker.Session()

role = get_execution_role()
print(role) # This is the role that SageMaker would use to leverage AWS resources (S3, CloudWatch) on your behalf

bucket = 'distribution-reliability-nlp'
print(bucket)

arn:aws:iam::435708786586:role/service-role/AmazonSageMaker-ExecutionRole-20190829T165674
distribution-reliability-nlp


In [10]:
outage = pd.read_csv('We_WPS_comments_categories.csv')
outage.head()

,outage category,Mobile Data Remarks,outage category number
0,Vegetation,1 LEG BROKE LOOSE at POLE DUE TO TREE LIMB PE...,5
1,Public,ON LtOut Haz WE8060 SAYS CREW NEEDED TO REPLAC...,4
2,Vegetation,ON HAZ WE9220 SAYS TREE LIMB FELL AND PULLED S...,5
3,Wildlife,ON LtOut TRBL SAYS SQUIRREL REPLACED FUSE,7
4,Equipment,CONNECTION BURNT OPEN AT SERVICE POLE OH CONN...,0


# Clean Data

In [39]:
outage_new_format = []
for i in range(0, len(outage)):
    outage_new_format.append('__label__' + str(outage.loc[i, 'outage category number']) + 
                             ' ' + outage.loc[i, 'Mobile Data Remarks'].lower() + '\n')

In [41]:
outage_new_format = ''.join(outage_new_format)

In [44]:
f = open("outage_new_format","w+")
f.write(outage_new_format)
f.close() 

In [46]:
#upload data to S3
sess.upload_data(path='outage_new_format', bucket=bucket)

's3://distribution-reliability-nlp/data/outage_new_format'

In [49]:
#where data is
s3_train_data = 's3://{}'.format(bucket)

In [50]:
#where model will be located
s3_output_location = 's3://{}'.format(bucket)

# Training

In [51]:
#Sagemaker training location
region_name = boto3.Session().region_name

In [52]:
#create container
container = sagemaker.amazon.amazon_estimator.get_image_uri(region_name, "blazingtext", "latest")
print('Using SageMaker BlazingText container: {} ({})'.format(container, region_name))

Using SageMaker BlazingText container: 825641698319.dkr.ecr.us-east-2.amazonaws.com/blazingtext:latest (us-east-2)


In [57]:
#set up model training 
bt_model = sagemaker.estimator.Estimator(container,
                                         role, 
                                         train_instance_count=1, 
                                         train_instance_type='ml.c4.4xlarge',
                                         train_volume_size = 30,
                                         train_max_run = 360000,
                                         input_mode= 'File',
                                         output_path=s3_output_location,
                                         sagemaker_session=sess)

In [54]:
#model hyperprams
bt_model.set_hyperparameters(mode="supervised",
                            epochs=10,
                            min_count=2,
                            learning_rate=0.05,
                            vector_dim=10,
                            early_stopping=True,
                            patience=4,
                            min_epochs=5,
                            word_ngrams=2)

In [55]:
#create handshake to start sagemaker session
train_data = sagemaker.session.s3_input(s3_train_data, distribution='FullyReplicated', 
                        content_type='text/plain', s3_data_type='S3Prefix')
data_channels = {'train': train_data}

In [58]:
#train the model
bt_model.fit(inputs=data_channels, logs=True)

2019-08-29 23:22:02 Starting - Starting the training job...
2019-08-29 23:22:05 Starting - Launching requested ML instances...
2019-08-29 23:23:02 Starting - Preparing the instances for training......
2019-08-29 23:23:47 Downloading - Downloading input data...
2019-08-29 23:24:22 Training - Training image download completed. Training in progress.
2019-08-29 23:24:22 Uploading - Uploading generated training model
2019-08-29 23:24:22 Failed - Training job failed

Arguments: train
[08/29/2019 23:24:16 WARNING 139921088612160] Loggers have already been setup.
[08/29/2019 23:24:16 WARNING 139921088612160] Loggers have already been setup.
[08/29/2019 23:24:16 INFO 139921088612160] nvidia-smi took: 0.0252630710602 secs to identify 0 gpus
[08/29/2019 23:24:16 ERROR 139921088612160] Customer Error: Please specify the 'mode' hyperparameter. Set it to 'supervised' for text classification or to one of ["skipgram", "cbow", "batch_skipgram"] for word2vec training.
Traceback (most recent call last):


UnexpectedStatusException: Error for Training job blazingtext-2019-08-29-23-22-02-592: Failed. Reason: ClientError: Please specify the 'mode' hyperparameter. Set it to 'supervised' for text classification or to one of ["skipgram", "cbow", "batch_skipgram"] for word2vec training.